In [31]:
%load_ext autoreload
%autoreload 2

from keys import *
from pathlib import Path
from urllib.parse import urlparse
from tinydb import TinyDB

import pinecone
import requests
from langchain import LLMChain, PromptTemplate
from langchain.chains import (ConversationalRetrievalChain,
                              SimpleSequentialChain)
from langchain.document_loaders import (PagedPDFSplitter, TextLoader,
                                        UnstructuredHTMLLoader,
                                        UnstructuredMarkdownLoader,
                                        UnstructuredPDFLoader)
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline, OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

from keys import HUGGINGFACE_HEADERS
from slackbot import chain

from pprint import PrettyPrinter
pp = PrettyPrinter()

# https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/pinecone.html?highlight=pinecone

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
class TestModel():
    def __init__(self):
        # llm = OpenAI()
       
        self.llm_model_id = "gpt2"
        # This model is pretty bad but using it for tests because it is free and
        # relatively small.

        # model_id = "decapoda-research/llama-7b-hf"
        # model_id = "decapoda-research/llama-13b-hf"

        self.model = AutoModelForCausalLM.from_pretrained(
            self.llm_model_id,
            device_map='auto',
            torch_dtype=torch.float16,
            local_files_only=True)

        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_id,
                                                       local_files_only=True)

        self.pipe = pipeline("text-generation",
                             model=self.model,
                             tokenizer=self.tokenizer,
                             max_new_tokens=16,
                             device_map="auto",
                             early_stopping=True)

        self.llm = HuggingFacePipeline(pipeline=self.pipe)

        template = """Q: {question} A:"""
        self.prompt = PromptTemplate(template=template, input_variables=["question"])
        self.llm_chain = LLMChain(prompt=self.prompt, llm=self.llm, verbose=True)

t = TestModel()

In [68]:
from tru_chain import TruChain, project, select

In [54]:
db = TinyDB("db.json")
tc = TruChain(t.llm_chain, db=db)
tc("hello there")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt after formatting:
Q: hello there A:


{'question': 'hello there',
 'text': " well thank you! (laughing) I mean, why wasn't I seen"}

In [85]:
tc._flush_records()
import tinydb
Record = tinydb.Query()
#for row in db.table("records").all():
#    print(project(address=("chain", "prompt", "template"), obj=row))

select(
    table=db.table("records"),
    fields=[
        ("chain","prompt","template"),
        ("chain", "_call", 0, "input", "inputs", "question"),
        ("chain", "_call", 0, "output", "text")
    ],
    where=Record.chain.prompt.template != None
)


,"(chain, prompt, template)","(chain, _call, 0, input, inputs, question)","(chain, _call, 0, output, text)"
0,Q: {question} A:,None,None
1,Q: {question} A:,None,None
2,Q: {question} A:,None,None
3,Q: {question} A:,hello there,"well thank you! (laughing) I mean, why wasn't..."


In [90]:
Record[0].something._path

(0, 'something')

In [63]:
Record.chain(row)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2322365/4239065856.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2322365/4239065856.py'                      │
│                                                                                                  │
│ /home/piotrm/anaconda3/envs/demo3/lib/python3.10/site-packages/tinydb/queries.py:92 in __call__  │
│                                                                                                  │
│    89 │   │   :param value: The value to check.                                                  │
│    90 │   │   :return: Whether the value matches this query.                                     │
│    91 │   │   """                                                                                │
│ ❱  92 │   │   return self._test(value)                                                           │
│    93 │                                                                                          │
│    94 │   def __hash__(self) -> int:                                                             │
│    95 │   │   # We calculate the query hash by using the ``hashval`` object which                │
│                                                                                                  │
│ /home/piotrm/anaconda3/envs/demo3/lib/python3.10/site-packages/tinydb/queries.py:175 in notest   │
│                                                                                                  │
│   172 │   │                                                                                      │
│   173 │   │   # Prevent empty queries to be evaluated                                            │
│   174 │   │   def notest(_):                                                                     │
│ ❱ 175 │   │   │   raise RuntimeError('Empty query was evaluated')                                │
│   176 │   │                                                                                      │
│   177 │   │   super().__init__(                                                                  │
│   178 │   │   │   test=notest,                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Empty query was evaluated

In [55]:
tc.records

[{'memory': None,
  'verbose': False,
  'chain': {'memory': None,
   'verbose': True,
   'prompt': {'input_variables': ['question'],
    'output_parser': None,
    'partial_variables': {},
    'template': 'Q: {question} A:',
    'template_format': 'f-string',
    'validate_template': True,
    '_type': 'prompt'},
   'llm': {'model_id': 'gpt2',
    'model_kwargs': None,
    '_type': 'huggingface_pipeline'},
   'output_key': 'text',
   '_type': 'llm_chain',
   '_call': [{'input': {'inputs': {'question': 'hello there'}},
     'start_time': '2023-05-07 22:20:59.713717',
     'end_time': '2023-05-07 22:20:59.888745',
     'pid': 2322365,
     'tid': 2322365,
     'chain_stack': [('chain',)],
     'output': {'text': " well thank you! (laughing) I mean, why wasn't I seen"}}]},
  '_type': 'TruChain'}]

In [46]:
tc._model

{'memory': None,
 'verbose': False,
 'chain': {'memory': None,
  'verbose': True,
  'prompt': {'input_variables': ['question'],
   'output_parser': None,
   'partial_variables': {},
   'template': 'Q: {question} A:',
   'template_format': 'f-string',
   'validate_template': True,
   '_type': 'prompt'},
  'llm': {'model_id': 'gpt2',
   'model_kwargs': None,
   '_type': 'huggingface_pipeline'},
  'output_key': 'text',
  '_type': 'llm_chain'},
 '_type': 'TruChain'}

In [37]:
db = TinyDB("db.json")
tab = db.table("records")
tab

<Table name='records', total=5, storage=<tinydb.storages.JSONStorage object at 0x7f442c68bd00>>

In [42]:
tc._flush_records(db)

<TinyDB tables=['records'], tables_count=1, default_table_documents_count=0, all_tables_documents_count=['records=5']>
<TinyDB tables=['records'], tables_count=1, default_table_documents_count=0, all_tables_documents_count=['records=5']>


In [43]:
if db:
    print(True)

In [ ]:
"""
    obj_id = id(obj)
    print(obj_id)

    if obj_id in self.encoded:
        return super().encode({'_ref': obj_id})
    
    # temporary reference
    self.encoded[obj_id] = {'_ref': obj_id}

    if isinstance(obj, Dict):
        ret = {}
        for k, v in obj.items():
            ret[k] = self.encode(v)

    elif isinstance(obj, Sequence):
        ret = []
        for v in obj:
            ret.append(self.encode(v))
    else:
        print(type(obj))
        ret = self.default(obj)
        # ret = super().encode(obj)

    self.encoded[obj_id] = ret

    return super().encode(ret)
"""

import langchain

class TestModel(pydantic.BaseModel):
    
    chain: langchain.chains.base.Chain


        
        #json_encoders = {
        #    TestModel: encoder.encode
        #}
        #json_dumps = lambda o, default: Config.encoder.encode(o)

LLMChain.Config = TestModel.Config
tm = TestModel(chain=t.llm_chain)

# json.dumps(tc.model, default=default)
"""
@jsonpickle.handlers.register(pydantic.BaseModel, base=True)
class FooHandler(jsonpickle.handlers.BaseHandler):
    def flatten(self, obj, data):
        print("pydantic", obj, data)
        h = jsonpickle.handlers.get(dict)#(self.context)
        print(h)
        h = h(self.context)
        h.flatten(obj.dict(), data)
        #if isinstance(obj, pydantic.BaseModel):
        #    # h = jsonpickle.handlers.BaseHandler(self.context)
        #    
        #    print(h)
        #    h.flatten(obj, data)
        #else:
        #    pass


@jsonpickle.handlers.register(object, base=True)
class FooHandler(jsonpickle.handlers.BaseHandler):
    def flatten(self, obj, data):
        print("default", obj, data)
        pass    
"""
        
# jsonpickle.handlers.register(object, )

#temp = jsonpickle.dumps(tc.chain)

# temp = jsonpickle.encode(tc.model)

# enc = MaybeJSONEncoder(check_circular=False)
# dec = MaybeJSONDecoder()

# dump = json.dumps(enc.encode(tc.chain))

# dec.decode(dump)
# print(dump)

In [ ]:
t.llm_chain.save("temp.json")
llm_chain2 = langchain.chains.loading.load_chain_from_config(t.llm_chain.dict())

In [ ]:
print(llm_chain2)

In [ ]:
print(type(tm))
encoded = tm.json(models_as_dict=False)
print(encoded)
TestModel.parse_raw(encoded)

In [ ]:
len(temp)

In [ ]:
tc.model_dict

In [ ]:
tc.records

In [ ]:
from tru_chain import Selection

tc._select(select=[
    Selection(param=("chain", "prompt", "template")),
    Selection(param=("chain", "llm", "model_id")),
    Selection(record=("input", "inputs", "question"))
])

In [ ]:
tc.model

In [ ]:
template = """Q: {question} A:"""
prompt = PromptTemplate(template=template,
                        input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=t.llm)

template_2 = """Reverse this sentence: {sentence}."""
prompt_2 = PromptTemplate(template=template_2,
                            input_variables=["sentence"])
llm_chain_2 = LLMChain(prompt=prompt_2, llm=t.llm)

seq_chain = SimpleSequentialChain(chains=[llm_chain, llm_chain_2],
                                    input_key="question",
                                    output_key="answer")

tc = TruChain(seq_chain)
tc._model

In [ ]:
tc("hello there")

In [ ]:
tc.records

In [ ]:
tc._get_obj_at_address(address=("chains", 0, ))

In [ ]:
tc.model

# Notes

1. Langchain does not have support for classification models: https://python.langchain.com/en/latest/modules/models.html

    - Will have to figure out out-of-band retrieval and execution of feedback models that are not LLM's.

2. Can add steps to chain to capture text at various points in a chain: https://python.langchain.com/en/latest/reference/modules/chains.html#langchain.chains.SequentialChain .


# Links

- https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/llama#transformers.LlamaForCausalLM

- https://huggingface.co/docs/transformers/main_classes/text_generation


# Pinecone



In [ ]:
from slackbot import chain
import langchain
import dill

In [ ]:
from langchain.chains import (ConversationalRetrievalChain,
                              SimpleSequentialChain)

verb = False

template = """Q: {question} A:"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=t.llm, verbose=verb)

template_2 = """Reverse this sentence: {sentence}."""
prompt_2 = PromptTemplate(template=template_2, input_variables=["sentence"])
llm_chain_2 = LLMChain(prompt=prompt_2, llm=t.llm, verbose=verb)

# print(llm_chain.run(question="What is the average air speed velocity of a laden swallow?"))

print(llm_chain_2.run(sentence="How are you doing?"))

seq_chain = SimpleSequentialChain(chains=[llm_chain, llm_chain_2], input_key="question", output_key="answer")
seq_chain.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
tru_chain_2 = TruChain(seq_chain)

In [ ]:
seq_chain.run(question="What is the average air speed velocity of a laden swallow? again")
tru_chain_2.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
for r in tru_chain_2.records:
    print(pp.pformat(r))

In [ ]:
tru_chain_2.run(question="What is the average air speed velocity of a laden swallow?")

In [ ]:
tru_chain_2.model

In [ ]:
import inspect
for frame_info in inspect.stack():
    frame = frame_info.frame
    print(frame_info.function)
    # print(frame.f_code)
    print(frame.f_locals.keys())

In [ ]:
db = TinyDB("db.json")
tab = db.table("records")

In [ ]:
tab = db.table("records")

In [ ]:
tab